In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

import plotly.graph_objects as go


In [3]:
vibe=pd.read_csv('vibe.csv')


In [4]:
vibe.columns


Index(['StoreID', 'OrderMode', 'WeekStartDate', 'WeeklyNetAmount', 'channel',
       'Brand', 'revenue'],
      dtype='object')

# High-Level Summary


<!-- Total E-commerce Revenue (Weekly)
% Change vs. Previous Week
% Change vs. Same Week Last Year
Top Performing Brand
Top Performing Channel
Overall Conversion Rate
Total Orders / AOV (Average Order Value) -->

In [61]:
66+24


90

In [60]:
vibe[vibe['Brand']=="LC"]['StoreID'].nunique()
vibe[vibe['Brand']=="WS"]['StoreID'].nunique()


74

In [31]:
vibe.columns


Index(['StoreID', 'OrderMode', 'WeekStartDate', 'WeeklyNetAmount', 'channel',
       'Brand', 'revenue'],
      dtype='object')

In [314]:
aux=vibe.groupby(['Brand','StoreID','OrderMode'])['revenue'].sum().reset_index()
aux['rev_%'] = (
    aux
      .groupby(['Brand'])['revenue']
      .transform(lambda x: x / x.sum() * 100)
)

# 3. (Optional) Round for readability
aux['rev_%'] = aux['rev_%'].round(1)
print(aux[aux['Brand']=="LC"]['rev_%'].sum())
aux.sort_values(by=['Brand','rev_%'],ascending=False)


100.2


,Brand,StoreID,OrderMode,revenue,rev_%
348,WS,1038,Online,1514902.42,1.9
539,WS,542,DoorDash,1364097.82,1.7
540,WS,542,Online,1302075.30,1.6
363,WS,117,Online,1196087.45,1.5
387,WS,1607,Online,1207734.77,1.5
...,...,...,...,...,...
326,LC,03646-00001,Grubhub,1589.37,0.0
328,LC,03646-00001,UberEats,6927.82,0.0
331,LC,03646-00002,Grubhub,4820.16,0.0
336,LC,03646-00003,Grubhub,6458.63,0.0


In [336]:
def less_rev_share_unit_Count(data):
        # data=aux.copy()
        # Step 1: Group stores under each channel as "X Stores"
        Brand=data.Brand.unique()[0]
        store_counts = data.groupby(['OrderMode'])['StoreID'].nunique().reset_index()
        store_counts['StoreGroup'] = store_counts['StoreID'].astype(str) + " Stores"

        # Step 2: Replace StoreID with grouped name
        data = data.merge(store_counts[['OrderMode', 'StoreGroup']], on='OrderMode', how='left')
        data['StoreID'] = data['StoreGroup']

        # Step 3: Create unique labels and map them to indices
        labels = pd.unique(data[['Brand', 'OrderMode', 'StoreID']].values.ravel()).tolist()
        label_map = {label: i for i, label in enumerate(labels)}

        # Step 4: Link from Brand → Channel
        link1 = data.groupby(['Brand', 'OrderMode']).size().reset_index(name='count')
        sources1 = link1['Brand'].map(label_map)
        targets1 = link1['OrderMode'].map(label_map)
        values1 = link1['count']

        # Step 5: Link from OrderMode → "X Stores"
        link2 = data.groupby(['OrderMode', 'StoreID']).size().reset_index(name='count')
        sources2 = link2['OrderMode'].map(label_map)
        targets2 = link2['StoreID'].map(label_map)
        values2 = link2['count']

        # Step 6: Combine all links
        sources = pd.concat([sources1, sources2])
        targets = pd.concat([targets1, targets2])
        values = pd.concat([values1, values2])

        # Step 7: Plot the Sankey diagram
        fig = go.Figure(data=[go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=labels,
                color="blue"
            ),
            link=dict(
                source=sources,
                target=targets,
                value=values
            )
        )])

        fig.update_layout(title_text=f"Count of Low-Performing Stores by Order Mode {Brand}", font_size=10)
        fig.show()
        



In [337]:
# revenue share calculated for each store and its channel 
# then flagging the store and channel with less revenue share 
# performance calculatuon at store and channel level 
# repeated stores because one storeid 2's doordash is in above 1, so counts too in above and storeis2 's online is less then 0 so also apears in < category


In [338]:
# select LC only
data=vibe[vibe['Brand']=="LC"]

# have clean df as each row represent store and its channel and sum of revenue
rev_share_order=data.groupby(['Brand','StoreID','OrderMode'])['revenue'].sum().reset_index()
# calculate rev share
rev_share_order['rev_%'] = (
    rev_share_order
      .groupby(['Brand'])['revenue']
      .transform(lambda x: x / x.sum() * 100)
)

# check the correctness of code
print(rev_share_order[rev_share_order['Brand']=="LC"]['rev_%'].sum())
rev_share_order.sort_values(by=['Brand','rev_%'],
                ascending=False,inplace=True)

rev_share_order['revenue'].sum()


100.0


np.float64(45054423.52999996)

In [339]:
rev_share_order['rev_%'].min()


np.float64(0.0004820170429112141)

In [340]:
rev_share_order['rev_%'].max()


np.float64(1.228648280521013)

In [341]:
import plotly.express as px

fig = px.histogram(
    data_frame=rev_share_order,
    x='rev_%',
    nbins=50,  # You can tweak this for granularity
    title='Distribution of Revenue Share(%) for StoreID-OrderMode combination',
    marginal='box' , # Adds a box plot on top for summary stats
     width=700,  # adjust width
    height=450 
)
fig.show()


In [342]:
# A. Percentile Thresholds
# Low performers: bottom 25% (Q1)

# High performers: top 25% (Q3)

q1 = rev_share_order['rev_%'].quantile(0.25)
q3 = rev_share_order['rev_%'].quantile(0.75)

def label_performance(value):
    if value <= q1:
        return 'Low'
    elif value >= q3:
        return 'High'
    else:
        return 'Medium'

rev_share_order['PerformanceLabel'] = rev_share_order['rev_%'].apply(label_performance)


In [343]:
rev_share_order['PerformanceLabel'].value_counts()


PerformanceLabel
Medium    172
High       86
Low        86
Name: count, dtype: int64

In [349]:
rev_share_order.groupby(['PerformanceLabel','OrderMode'])['StoreID'].count().reset_index()


,PerformanceLabel,OrderMode,StoreID
0,High,DoorDash,9
1,High,Mobile,61
2,High,Website,16
3,Low,Grubhub,40
4,Low,UberEats,46
5,Medium,DoorDash,67
6,Medium,Grubhub,3
7,Medium,Mobile,15
8,Medium,UberEats,27
9,Medium,Website,60


In [346]:


aux=rev_share_order[rev_share_order['PerformanceLabel']=='Low']


less_rev_share_unit_Count(aux)


In [269]:
data.head(2)


,Brand,StoreID,OrderMode,revenue,rev_%
1,LC,01389-00001,Grubhub,1222.01,0.0
3,LC,01389-00001,UberEats,18098.01,0.0


In [307]:
# LC's Ubereats and Grubhub shares almost =0 for 53, and 41 stores respectively
# let's find stores

lc_location=pd.read_excel('VIBE_LC locations.xlsx')
# Ensure the column is float (if it isn't already)
lc_location['Franchise Store'] = lc_location['Franchise Store'].astype(float)

# Split the number into integer and decimal parts
lc_location['store_id_formatted'] =lc_location['Franchise Store'] .apply(
    lambda x: f"{int(x):05d}-{int(round((x - int(x)) * 10000)):05d}"
)
lc_location['store_id_formatted'].unique()

df=lc_location[['ST','store_id_formatted','City','Zone','Latitude', 'Longitude']]
df['StoreID']=df['store_id_formatted']

lc_extended=rev_share_order.merge(df,on='StoreID',how='left')
lc_extended.head()


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_15637/1887137023.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Brand,StoreID,OrderMode,revenue,rev_%,PerformanceLabel,ST,store_id_formatted,City,Zone,Latitude,Longitude
0,LC,03222-00036,Mobile,553560.40,1.228648,High,NC,03222-00036,Charlotte,3222,35.212489,-80.690501
1,LC,03222-00026,Mobile,548029.18,1.216372,High,KS,03222-00026,Manhattan,3222,39.191931,-96.608682
2,LC,03222-00015,Mobile,521593.97,1.157698,High,NM,03222-00015,Hobbs,3222,32.726233,-103.143932
3,LC,01826-00002,Mobile,435204.57,0.965953,High,OK,01826-00002,Durant,1826,33.998580,-96.411164
4,LC,01893-00001,Mobile,427600.27,0.949075,High,TX,01893-00001,Portland,1893,27.890488,-97.310698


In [350]:
# Step 3: Plot with Plotly Express
fig = px.scatter_mapbox(
    lc_extended[lc_extended['PerformanceLabel']=='Low'],
    lat='Latitude',
    lon='Longitude',
    color='OrderMode',
    hover_name='StoreID',
    hover_data=['StoreID'],
    zoom=4,
    mapbox_style='carto-positron',
    title='Franchise Stores Below 25th Percentile (Low Performing)',
    color_discrete_map={True: 'red', False: 'blue'},
    height=600
)
fig.show()


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_15637/1013503544.py:2: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [309]:
top=lc_extended[lc_extended['PerformanceLabel']=="High"].sort_values(by='rev_%',ascending=False)[:3]
top


,Brand,StoreID,OrderMode,revenue,rev_%,PerformanceLabel,ST,store_id_formatted,City,Zone,Latitude,Longitude
0,LC,03222-00036,Mobile,553560.40,1.228648,High,NC,03222-00036,Charlotte,3222,35.212489,-80.690501
1,LC,03222-00026,Mobile,548029.18,1.216372,High,KS,03222-00026,Manhattan,3222,39.191931,-96.608682
2,LC,03222-00015,Mobile,521593.97,1.157698,High,NM,03222-00015,Hobbs,3222,32.726233,-103.143932


In [353]:
fig = px.scatter_mapbox(
    top,
    lat='Latitude',
    lon='Longitude',
    color='OrderMode',
    hover_name='StoreID',
    hover_data=['StoreID'],
    zoom=4,
    mapbox_style='carto-positron',
    title='Top-Performing Franchise Stores (Above 95th Percentile)',
    color_discrete_map={True: 'green', False: 'red'},
    height=600
)
fig.show()


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_15637/2290187449.py:1: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [356]:
top[['StoreID','OrderMode','ST','City','rev_%']]


,StoreID,OrderMode,ST,City,rev_%
0,03222-00036,Mobile,NC,Charlotte,1.228648
1,03222-00026,Mobile,KS,Manhattan,1.216372
2,03222-00015,Mobile,NM,Hobbs,1.157698


In [119]:

data=aux[aux['Brand']=="WS"]

less_rev_share_unit_Count(data)



# last week analysis


In [389]:
def negative_last_two_weeks(data):
   
    # Sort data
    data = data.sort_values(by=['StoreID', 'WeekStartDate'])

    # Get last two unique week dates
    latest_dates = data['WeekStartDate'].drop_duplicates().sort_values(ascending=False).unique()
    last_two_weeks = latest_dates[:2]
    print(last_two_weeks)
    # Filter recent data for these two weeks
    recent_data = data[data['WeekStartDate'].isin(last_two_weeks)]

    # Since Brand and OrderMode are at store level (assuming they don't change weekly),
    # get unique Brand and OrderMode per StoreID
    store_info = data[['StoreID', 'Brand', 'OrderMode']].drop_duplicates(subset=['StoreID'])

    # Pivot to get Week_1 and Week_2 WeeklyNetAmount side by side
    pivoted = recent_data.pivot_table(index='StoreID',
                                    columns='WeekStartDate',
                                    values='WeeklyNetAmount')

    # Rename columns for clarity if exactly two weeks exist
    if pivoted.shape[1] == 2:
        pivoted.columns = ['week_1_rev', 'week_2_rev']

    # Calculate percentage change
    pivoted['PctChange'] = ((pivoted['week_2_rev'] - pivoted['week_1_rev']) / pivoted['week_1_rev']) * 100

    # Filter for negative change stores
    negative_change_stores = pivoted[pivoted['PctChange'] < 0].reset_index()

    # Merge Brand and OrderMode info back
    negative_change_stores = negative_change_stores.merge(store_info, on='StoreID', how='left')

    # Reorder columns for clarity
    final_columns = ['StoreID', 'Brand', 'OrderMode', 'week_1_rev', 'week_2_rev', 'PctChange']
    negative_change_stores = negative_change_stores[final_columns]

    negative_change_stores
    negative_change_stores['week_1']=last_two_weeks[1]
    negative_change_stores['week_2']=last_two_weeks[0]
    # negative_change_stores['week1']=last_two_weeks[0]
    # negative_change_stores['week2']=last_two_weeks[1]
    return negative_change_stores



In [468]:
data=vibe.copy()
data=data[data['Brand']=="LC"]

# Make sure WeekStartDate is a datetime
data['WeekStartDate'] = pd.to_datetime(data['WeekStartDate'])

# Sort data by StoreID and WeekStartDate
data = data.sort_values(by=['StoreID', 'WeekStartDate'])

# Group by Store (or Brand, Channel if needed) and calculate % change
data['PctChangeVsPrevWeek'] = data.groupby(['Brand','StoreID','OrderMode'])['WeeklyNetAmount'].pct_change() * 100
# data[(data['StoreID']=='01389-00005')&(data['OrderMode']=='Website')  & (data['WeekStartDate'].isin(['2025-04-21','2025-04-14']))]

print(data.shape)
latest_dates = data['WeekStartDate'].drop_duplicates().sort_values(ascending=False).unique()
last_two_weeks = latest_dates[:2]
last_two_weeks

recent_data = data[data['WeekStartDate'].isin(last_two_weeks)]
# recent_data[(recent_data['StoreID']=='01389-00005')&(recent_data['OrderMode']=='Website')  & (recent_data['WeekStartDate'].isin(['2025-04-21','2025-04-14']))]

recent_data=recent_data[recent_data['PctChangeVsPrevWeek']<=-30]
recent_data
print(recent_data.shape)
recent_data
recent_data[(recent_data['StoreID']=='01389-00005')&(recent_data['OrderMode']=='Website')  & (recent_data['WeekStartDate'].isin(['2025-04-21','2025-04-14']))]


(19945, 8)
(53, 8)


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_15637/1734134436.py:26: FutureWarning:

The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.



,StoreID,OrderMode,WeekStartDate,WeeklyNetAmount,channel,Brand,revenue,PctChangeVsPrevWeek


In [465]:
# anotherway

data=vibe.copy()

# Make sure WeekStartDate is a datetime
data['WeekStartDate'] = pd.to_datetime(data['WeekStartDate'])

# Sort data by StoreID and WeekStartDate
data = data.sort_values(by=['StoreID', 'WeekStartDate'])

# Group by Store (or Brand, Channel if needed) and calculate % change
data['PctChangeVsPrevWeek'] = data.groupby(['Brand','StoreID','OrderMode'])['WeeklyNetAmount'].pct_change() * 100
# data[(data['StoreID']=='01389-00005')&(data['OrderMode']=='Website')  & (data['WeekStartDate'].isin(['2025-04-21','2025-04-14']))]


lc_neg=negative_last_two_weeks(data[data['Brand']=="LC"])

lc_neg.groupby('OrderMode')['StoreID'].count()


# flagging store-channel combination that has negative 30%bdecline
lc_neg=lc_neg[lc_neg['PctChange']<=-30]
lc_neg.shape


<DatetimeArray>
['2025-04-21 00:00:00', '2025-04-14 00:00:00']
Length: 2, dtype: datetime64[ns]


(2, 8)

,StoreID,Brand,OrderMode,Week_1,Week_2,PctChange,week_1,week_2
7,01494-00001,LC,UberEats,2559.34,1731.350,-32.351700,2025-04-21,2025-04-14
18,01944-00004,LC,Mobile,2129.86,1476.406,-30.680608,2025-04-21,2025-04-14


In [365]:

# ws_neg=negative_last_two_weeks(data[data['Brand']=="WS"])
# ws_neg.groupby('OrderMode')['StoreID'].count()



In [311]:
# order analysis


In [5]:
lc_weekly_order=pd.read_csv('./weekly_sales/weekly_orders.csv')

# lc_weekly_sale by 1po_3po
first_party=['Mobile','Website']
third_party=['DoorDash', 'UberEats','Grubhub']


lc_weekly_order['channel']=np.where(lc_weekly_order['OrderPlacedMethod'].isin(first_party),'1PO',np.where(lc_weekly_order['OrderPlacedMethod'].isin(third_party),'3PO',lc_weekly_order['OrderPlacedMethod']))
lc_weekly_order=lc_weekly_order[lc_weekly_order['channel'].isin(['1PO','3PO'])]
lc_weekly_order.dtypes
lc_weekly_order['Brand']="LC"


In [187]:
lc_weekly_order.head(5)


,FranchiseStore,OrderPlacedMethod,WeeklyRevenue,WeekStartDate,UniqueOrderCount,channel,Brand
1,01389-00001,DoorDash,775.89,2024-02-19,41,3PO,LC
2,01389-00001,DoorDash,2017.23,2024-02-26,116,3PO,LC
3,01389-00001,DoorDash,2003.98,2024-03-04,109,3PO,LC
4,01389-00001,DoorDash,2250.47,2024-03-11,125,3PO,LC
5,01389-00001,DoorDash,2533.61,2024-03-18,147,3PO,LC


#  in last 3 month store-mode combinatoions with consective declineing AOA 


In [471]:
import pandas as pd



data=lc_weekly_order.copy()

# Calculate AOV (Average Order Value)
data['AOV'] = data['WeeklyRevenue'] / data['UniqueOrderCount']

# Sort for proper pct_change calculation
data = data.sort_values(['OrderPlacedMethod', 'WeekStartDate'])
data
data['WeekStartDate'] = pd.to_datetime(data['WeekStartDate'])
# Calculate 3 months ago from today
three_months_ago = pd.Timestamp.today() - pd.DateOffset(months=3)

# Filter the data
data = data[data['WeekStartDate'] >= three_months_ago]


In [472]:
# pct_change() computes (current - previous) / previous * 100
# Calculate Week-over-Week Revenue Growth % per Brand  OrderPlacedMethod
data['WoW_Growth_Percent'] = data.groupby(['FranchiseStore','OrderPlacedMethod'])['WeeklyRevenue'].pct_change() * 100

# Round for presentation
data['WeeklyRevenue'] = data['WeeklyRevenue'].round(2)
data['AOV'] = data['AOV'].round(2)
data['WoW_Growth_Percent'] = data['WoW_Growth_Percent'].round(2)

# data[(data['FranchiseStore']=='01389-00001')&(data['OrderPlacedMethod']=='DoorDash')][['WeekStartDate','OrderPlacedMethod','WeeklyRevenue','WoW_Growth_Percent']]
data[data['WoW_Growth_Percent']<=-30]


,FranchiseStore,OrderPlacedMethod,WeeklyRevenue,WeekStartDate,UniqueOrderCount,channel,Brand,AOV,WoW_Growth_Percent
18197,03222-00008,DoorDash,332.10,2025-03-03,17,3PO,LC,19.54,-82.65
17793,03222-00007,DoorDash,1232.26,2025-03-10,61,3PO,LC,20.20,-36.40
19031,03222-00010,DoorDash,572.15,2025-03-10,32,3PO,LC,17.88,-47.58
8119,01893-00001,DoorDash,1349.42,2025-03-17,63,3PO,LC,21.42,-37.17
9959,01940-00009,DoorDash,1614.90,2025-03-17,78,3PO,LC,20.70,-36.60
...,...,...,...,...,...,...,...,...,...
11329,01940-00011,Website,1920.95,2025-04-21,84,1PO,LC,22.87,-41.85
13703,01944-00004,Website,2178.87,2025-04-21,91,1PO,LC,23.94,-32.64
5189,01494-00001,Website,2006.79,2025-05-05,88,1PO,LC,22.80,-45.22
11331,01940-00011,Website,2011.11,2025-05-05,86,1PO,LC,23.39,-30.43


In [473]:
import pandas as pd

# Step 1: Make sure your data is sorted correctly
data['WeekStartDate'] = pd.to_datetime(data['WeekStartDate'])
data = data.sort_values(['FranchiseStore', 'OrderPlacedMethod', 'WeekStartDate'])

# Step 2: Calculate WoW_Growth_Percent (if not already present or needs recalculating)
data['WoW_Growth_Percent'] = (
    data.groupby(['FranchiseStore', 'OrderPlacedMethod'])['WeeklyRevenue']
    .pct_change() * 100
)

# Step 3: Detect 4 consecutive declines
def has_4_consecutive_declines(group):
    # Create a boolean Series: True if WoW_Growth_Percent < 0
    decline = group['WoW_Growth_Percent'] < 0
    # Use rolling window of size 4 over the boolean to check for 4 Trues in a row
    decline_4 = decline.rolling(window=4).apply(lambda x: all(x), raw=True)
    return decline_4.any()

# Step 4: Group and filter those with 4 consecutive declines
declining_groups = (
    data.groupby(['FranchiseStore', 'OrderPlacedMethod'])
    .filter(has_4_consecutive_declines)
    [['FranchiseStore', 'OrderPlacedMethod']]
    .drop_duplicates()
)

data[(data['FranchiseStore']=='01389-00001')&(data['OrderPlacedMethod']=='Mobile')][['WeekStartDate','OrderPlacedMethod','WeeklyRevenue','WoW_Growth_Percent']]
declining_groups



,FranchiseStore,OrderPlacedMethod
5645,01494-00002,Website
6498,01494-00005,Website
14071,01944-00005,Website
14515,01978-00002,DoorDash
16288,03222-00002,UberEats
16938,03222-00004,Grubhub
21124,03222-00016,Website
23221,03222-00022,UberEats
23348,03222-00023,DoorDash
24554,03222-00026,DoorDash


In [474]:
import pandas as pd
import plotly.express as px

# find store with their ordermode declining 4 conssective weeks Week-over-Week Revenue Growth % per Brand  OrderPlacedMethod
data['WeekStartDate'] = pd.to_datetime(data['WeekStartDate'])
three_months_ago = pd.Timestamp.today() - pd.DateOffset(months=3)
recent_data = data[data['WeekStartDate'] >= three_months_ago].copy()

# --- Step 2: Calculate WoW_Growth_Percent ---
recent_data = recent_data.sort_values(['FranchiseStore', 'OrderPlacedMethod', 'WeekStartDate'])
recent_data['WoW_Growth_Percent'] = (
    recent_data.groupby(['FranchiseStore', 'OrderPlacedMethod'])['WeeklyRevenue']
    .pct_change() * 100
)

# --- Step 3: Identify 4 consecutive declines ---
def has_4_consecutive_declines(group):
    decline = group['WoW_Growth_Percent'] < 0
    rolling = decline.rolling(window=4).apply(lambda x: all(x), raw=True)
    return rolling.any()

declining_keys = (
    recent_data.groupby(['FranchiseStore', 'OrderPlacedMethod'])
    .filter(has_4_consecutive_declines)
    [['FranchiseStore', 'OrderPlacedMethod']]
    .drop_duplicates()
)

# --- Step 4: Filter to only declining groups ---
declining_data = pd.merge(recent_data, declining_keys, on=['FranchiseStore', 'OrderPlacedMethod'])

# --- Step 5: Plot with Plotly Express ---
fig = px.line(
    declining_data,
    x='WeekStartDate',
    y='WeeklyRevenue',
    color='FranchiseStore',
    facet_col='OrderPlacedMethod',
    line_group='FranchiseStore',
    markers=True,
    title='Franchises with 4-Week Revenue Declines (Last 3 Months)'
)

fig.update_layout(
    height=600,
    legend_title_text='FranchiseStore',
    xaxis_title='Week Start Date',
    yaxis_title='Weekly Revenue'
)

fig.update_xaxes(tickangle=45)
fig.show()


#  AOA  analysis


In [259]:
# Understand which OrderPlacedMethod (your ecommerce channels) deliver:

# High Average Order Value (AOV) → more value per order
# High Order Volume → more customer activity
# This helps identify:
# Channels bringing in high-value customers
# Channels with high traffic but potentially low spend per order (opportunity for upsell)


In [490]:
data=lc_weekly_order.copy()
data['WeekStartDate'] = pd.to_datetime(data['WeekStartDate'])
# Calculate AOV (Average Order Value)

data['AOV'] = data['WeeklyRevenue'] / data['UniqueOrderCount']
three_months_ago = pd.Timestamp.today() - pd.DateOffset(months=3)

# Filter the data for last 3 months
data = data[data['WeekStartDate'] >= three_months_ago]
data.head()


,FranchiseStore,OrderPlacedMethod,WeeklyRevenue,WeekStartDate,UniqueOrderCount,channel,Brand,AOV
54,01389-00001,DoorDash,3844.97,2025-02-24,203,3PO,LC,18.940739
55,01389-00001,DoorDash,3119.65,2025-03-03,167,3PO,LC,18.680539
56,01389-00001,DoorDash,3007.84,2025-03-10,152,3PO,LC,19.788421
57,01389-00001,DoorDash,3410.18,2025-03-17,184,3PO,LC,18.533587
58,01389-00001,DoorDash,3242.99,2025-03-24,169,3PO,LC,19.189290


In [491]:
data['WoW_Growth_Percent'] = data.groupby(['FranchiseStore','OrderPlacedMethod'])['WeeklyRevenue'].pct_change() * 100



# Round for presentation
data['WeeklyRevenue'] = data['WeeklyRevenue'].round(2)
data['AOV'] = data['AOV'].round(2)
data['WoW_Growth_Percent'] = data['WoW_Growth_Percent'].round(2)
data.head()
print(data.shape)
data.head()


(3558, 9)


,FranchiseStore,OrderPlacedMethod,WeeklyRevenue,WeekStartDate,UniqueOrderCount,channel,Brand,AOV,WoW_Growth_Percent
54,01389-00001,DoorDash,3844.97,2025-02-24,203,3PO,LC,18.94,NaN
55,01389-00001,DoorDash,3119.65,2025-03-03,167,3PO,LC,18.68,-18.86
56,01389-00001,DoorDash,3007.84,2025-03-10,152,3PO,LC,19.79,-3.58
57,01389-00001,DoorDash,3410.18,2025-03-17,184,3PO,LC,18.53,13.38
58,01389-00001,DoorDash,3242.99,2025-03-24,169,3PO,LC,19.19,-4.90


In [494]:
# Group by Store and OrderPlacedMethod
conversion_quality = data.groupby(['FranchiseStore', 'OrderPlacedMethod']).agg({
    'UniqueOrderCount': 'sum',
    'WeeklyRevenue': 'sum',
    'AOV': 'mean'
}).reset_index()
print(conversion_quality.shape)

conversion_quality


(340, 5)


,FranchiseStore,OrderPlacedMethod,UniqueOrderCount,WeeklyRevenue,AOV
0,01389-00001,DoorDash,1987,37084.17,18.683636
1,01389-00001,Grubhub,12,236.08,21.245000
2,01389-00001,Mobile,5138,93774.63,18.296364
3,01389-00001,UberEats,174,3519.83,20.276364
4,01389-00001,Website,2129,45241.00,21.339091
...,...,...,...,...,...
335,03646-00003,Website,2315,47872.85,20.729091
336,03646-00004,DoorDash,465,9169.94,19.671818
337,03646-00004,Mobile,1230,22667.16,18.508182
338,03646-00004,UberEats,9,146.47,16.274444


In [495]:




# Optional: Add Revenue Per Order
conversion_quality['RevenuePerOrder'] = (
    conversion_quality['WeeklyRevenue'] / conversion_quality['UniqueOrderCount']
).round(2)

# Round values
conversion_quality['AOV'] = conversion_quality['AOV'].round(2)
conversion_quality['WeeklyRevenue'] = conversion_quality['WeeklyRevenue'].round(2)

# Sort by AOV or Revenue
conversion_quality = conversion_quality.sort_values(
    by=['FranchiseStore', 'AOV'], ascending=[True, False]
)

conversion_quality
conversion_quality[(conversion_quality['FranchiseStore']=='01389-00001')&(conversion_quality['OrderPlacedMethod']=='Mobile')]

conversion_quality.shape
conversion_quality.head()


,FranchiseStore,OrderPlacedMethod,UniqueOrderCount,WeeklyRevenue,AOV,RevenuePerOrder
4,01389-00001,Website,2129,45241.00,21.34,21.25
1,01389-00001,Grubhub,12,236.08,21.24,19.67
3,01389-00001,UberEats,174,3519.83,20.28,20.23
0,01389-00001,DoorDash,1987,37084.17,18.68,18.66
2,01389-00001,Mobile,5138,93774.63,18.30,18.25


In [496]:
# Use quantiles to define performance bands
revenue_80 = conversion_quality['WeeklyRevenue'].quantile(0.80)
aov_80 = conversion_quality['AOV'].quantile(0.80)

revenue_20 = conversion_quality['WeeklyRevenue'].quantile(0.20)
aov_20 = conversion_quality['AOV'].quantile(0.20)


In [497]:
top_performers = conversion_quality[
    (conversion_quality['WeeklyRevenue'] >= revenue_80) &
    (conversion_quality['AOV'] >= aov_80)
].sort_values(by=['WeeklyRevenue'], ascending=False)

print("Top-Performing Store + Channel Combos")
top_performers


Top-Performing Store + Channel Combos


,FranchiseStore,OrderPlacedMethod,UniqueOrderCount,WeeklyRevenue,AOV,RevenuePerOrder
214,03222-00017,Mobile,3484,78607.88,22.54,22.56
109,01940-00011,Mobile,2549,57880.27,22.79,22.71
208,03222-00015,Website,2337,55711.93,23.84,23.84
216,03222-00017,Website,1820,44643.89,24.52,24.53


In [498]:
underperformers = conversion_quality[
    (conversion_quality['WeeklyRevenue'] <= revenue_20) &
    (conversion_quality['AOV'] <= aov_20)
].sort_values(by=['WeeklyRevenue'])

print("Underperforming Store + Channel Combos")
underperformers


Underperforming Store + Channel Combos


,FranchiseStore,OrderPlacedMethod,UniqueOrderCount,WeeklyRevenue,AOV,RevenuePerOrder
222,03222-00019,Grubhub,2,33.42,16.71,16.71
205,03222-00015,Grubhub,7,143.73,17.78,20.53
338,03646-00004,UberEats,9,146.47,16.27,16.27
224,03222-00019,UberEats,10,169.88,15.30,16.99
195,03222-00012,DoorDash,12,212.39,17.70,17.70
322,03646-00001,Grubhub,11,213.93,18.18,19.45
200,03222-00013,Website,12,215.03,17.92,17.92
196,03222-00012,Mobile,17,311.70,18.34,18.34
32,01389-00011,Grubhub,24,391.14,15.93,16.30
248,03222-00026,Grubhub,27,485.02,18.06,17.96


In [499]:
def categorize(row):
    if row['WeeklyRevenue'] >= revenue_80 and row['AOV'] >= aov_80:
        return 'Top Performer'
    elif row['WeeklyRevenue'] <= revenue_20 and row['AOV'] <= aov_20:
        return 'Underperformer'
    else:
        return 'Average'

conversion_quality['Performance'] = conversion_quality.apply(categorize, axis=1)


In [500]:
# | Group               | Insight                                             | Action                                    |
# | ------------------- | --------------------------------------------------- | ----------------------------------------- |
# | **Top Performers**  | Channels/stores that are working exceptionally well | Invest more, replicate strategy           |
# | **Underperformers** | Low-value orders, weak volume or both               | Investigate & optimize: UX, pricing, etc. |


In [501]:
import plotly.express as px

# Create a label combining Store and Channel for hover info
conversion_quality['Label'] = conversion_quality['FranchiseStore'] + " | " + conversion_quality['OrderPlacedMethod']

fig = px.scatter(
    conversion_quality,
    x='AOV',
    y='WeeklyRevenue',
    size='UniqueOrderCount',
    color='Performance',
    hover_name='Label',
    title='Store + Channel Performance (AOV vs Revenue)',
    labels={
        'AOV': 'Average Order Value',
        'WeeklyRevenue': 'Total Revenue',
        'UniqueOrderCount': 'Order Volume'
    },
    size_max=40,
    color_discrete_map={
        'Top Performer': 'green',
        'Underperformer': 'red',
        'Average': 'gray'
    }
)

fig.update_layout(
    template='plotly_white',
    legend_title_text='Performance Category'
)

fig.show()


In [508]:
conversion_quality['Performance'].unique()


array(['Average', 'Underperformer', 'Top Performer'], dtype=object)

In [506]:
conversion_quality[conversion_quality['Performance']=='Topperformer']


,FranchiseStore,OrderPlacedMethod,UniqueOrderCount,WeeklyRevenue,AOV,RevenuePerOrder,Performance,Label


In [505]:
conversion_quality[conversion_quality['Performance']=='Average']


,FranchiseStore,OrderPlacedMethod,UniqueOrderCount,WeeklyRevenue,AOV,RevenuePerOrder,Performance,Label
4,01389-00001,Website,2129,45241.00,21.34,21.25,Average,01389-00001 | Website
1,01389-00001,Grubhub,12,236.08,21.24,19.67,Average,01389-00001 | Grubhub
3,01389-00001,UberEats,174,3519.83,20.28,20.23,Average,01389-00001 | UberEats
0,01389-00001,DoorDash,1987,37084.17,18.68,18.66,Average,01389-00001 | DoorDash
2,01389-00001,Mobile,5138,93774.63,18.30,18.25,Average,01389-00001 | Mobile
...,...,...,...,...,...,...,...,...
333,03646-00003,Mobile,4416,85897.91,19.49,19.45,Average,03646-00003 | Mobile
331,03646-00003,DoorDash,1617,30729.47,19.03,19.00,Average,03646-00003 | DoorDash
339,03646-00004,Website,573,11563.93,20.05,20.18,Average,03646-00004 | Website
336,03646-00004,DoorDash,465,9169.94,19.67,19.72,Average,03646-00004 | DoorDash


In [509]:
conversion_quality[conversion_quality['Performance']=='Top Performer']


,FranchiseStore,OrderPlacedMethod,UniqueOrderCount,WeeklyRevenue,AOV,RevenuePerOrder,Performance,Label
109,01940-00011,Mobile,2549,57880.27,22.79,22.71,Top Performer,01940-00011 | Mobile
208,03222-00015,Website,2337,55711.93,23.84,23.84,Top Performer,03222-00015 | Website
216,03222-00017,Website,1820,44643.89,24.52,24.53,Top Performer,03222-00017 | Website
214,03222-00017,Mobile,3484,78607.88,22.54,22.56,Top Performer,03222-00017 | Mobile


In [513]:
import plotly.express as px

# Assume your DataFrame is named df and has columns:
# 'WeeklyRevenue', 'AOV', 'label', and 'Performance'

fig = px.scatter(
   conversion_quality,
    x="WeeklyRevenue",
    y="AOV",
    color="Label",
    facet_col="Performance",
    title="AOV vs Weekly Revenue by Performance Group",
    labels={
        "WeeklyRevenue": "Weekly Revenue",
        "AOV": "Average Order Value",
        "label": "Label",
        "Performance": "Performance Group"
    },
    template="plotly_white"
)

fig.update_traces(marker=dict(size=10, opacity=0.7), selector=dict(mode='markers'))
fig.update_layout(margin=dict(t=60, l=30, r=30, b=30))
fig.show()


In [512]:
import plotly.express as px

# Assume your DataFrame is named df and has columns:
# 'WeeklyRevenue', 'AOV', 'label', and 'Performance'

fig = px.scatter(
   conversion_quality,
    x="WeeklyRevenue",
    y="AOV",
    color="Label",

    title="AOV vs Weekly Revenue by Performance Group",
    labels={
        "WeeklyRevenue": "Weekly Revenue",
        "AOV": "Average Order Value",
        "label": "Label",
        "Performance": "Performance Group"
    },
    template="plotly_white"
)

fig.update_traces(marker=dict(size=10, opacity=0.7), selector=dict(mode='markers'))
fig.update_layout(margin=dict(t=60, l=30, r=30, b=30))
fig.show()
